### Дипломная работа Н.В.Чепурных, студент курса "Аналитик данных"
### Научный руководитель: Алексей Кузьмин
Тема: прогнозирование ежемесячного потребления электрической энергии физическими лицами в электросетевой компании.

Этапы работ:
- Этап 1. Загрузка и объединение данных
- Этап 2. Обработка данных
- Этап 3. Применение модели Линейной регрессии

Загрузка библиотек:

In [1]:
import os
import pandas as pd
import re

### Этап 1. Загрузка и объединение данных

1.1. Загрузка данных по полезному отпуску электроэнергии за период 2019-2020

In [2]:
%%time
dirName = './Data/Supply/'
supply_all = []
for name in os.listdir(dirName):
    fullname = os.path.join(dirName, name) # получаем полное имя
    if os.path.isfile(fullname):
        df = pd.read_excel(fullname)
        print ("Обработан файл:", fullname, " - загружено строк:", len(df))
        supply_all.append(df)
supply_temp = pd.concat(supply_all,axis=0,ignore_index=True)
print("Всего строк загружено: ", len(supply_temp))

Обработан файл: ./Data/Supply/2019 01.xlsx  - загружено строк: 361365
Обработан файл: ./Data/Supply/2019 02.xlsx  - загружено строк: 361384
Обработан файл: ./Data/Supply/2019 03.xlsx  - загружено строк: 361888
Обработан файл: ./Data/Supply/2019 04.xlsx  - загружено строк: 368771
Обработан файл: ./Data/Supply/2019 05.xlsx  - загружено строк: 373635
Обработан файл: ./Data/Supply/2019 06.xlsx  - загружено строк: 377873
Обработан файл: ./Data/Supply/2019 07.xlsx  - загружено строк: 377808
Обработан файл: ./Data/Supply/2019 08.xlsx  - загружено строк: 376587
Обработан файл: ./Data/Supply/2019 09.xlsx  - загружено строк: 374118
Обработан файл: ./Data/Supply/2019 10.xlsx  - загружено строк: 379904
Обработан файл: ./Data/Supply/2019 11.xlsx  - загружено строк: 375713
Обработан файл: ./Data/Supply/2019 12.xlsx  - загружено строк: 370133
Обработан файл: ./Data/Supply/2020 01.xlsx  - загружено строк: 360223
Обработан файл: ./Data/Supply/2020 02.xlsx  - загружено строк: 362710
Обработан файл: ./Da

In [3]:
supply_temp

,Method,Consumer,Building,Supply,Status,Period
0,ЛС-СДСД,66700000233,дача,0.0,неоспар.,01-2019
1,ЛС-СДСД,66700000410,дача,0.0,неоспар.,01-2019
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,неоспар.,01-2019
3,ЛС-СДСД,66700000557,дача,2.0,неоспар.,01-2019
4,ЛС-СДСД,66700000575,дача,21.0,неоспар.,01-2019
...,...,...,...,...,...,...
5545974,ЛС-ЭНОРМ,69198110501,дача,0.0,неоспар.,03-2020
5545975,ЛС-ЭНОРМ,69198110514,дача,145.0,неоспар.,03-2020
5545976,ЛС-ЭНОРМ,69198110624,дача,145.0,неоспар.,03-2020
5545977,ЛС-ЭНОРМ,69198110718,дача,145.0,неоспар.,03-2020


1.2. Загрузка данных по погоде с 2018 года  по апрель 2020 года.

Сайт с архивом погоды: https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%9F%D0%B5%D1%80%D0%BC%D0%B8,_%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%BC_%D0%A1%D0%B0%D0%B2%D0%B8%D0%BD%D0%BE_(%D0%B0%D1%8D%D1%80%D0%BE%D0%BF%D0%BE%D1%80%D1%82),_METAR

In [4]:
%%time
dirName = './Data/'
name = 'perm_weather_2018.csv'
fullname = os.path.join(dirName, name) # получаем полное имя
if os.path.isfile(fullname):
    weather_temp = pd.read_csv(fullname, sep = ';', encoding = "ISO-8859-1", engine='python')
    print ("Обработан файл:", fullname, " - загружено строк:", len(weather_temp))
else: 
    print ("Ошибка загрузки:", fullname)

Обработан файл: ./Data/perm_weather_2018.csv  - загружено строк: 39987
Wall time: 472 ms


In [5]:
weather_temp.head()

,local time Perm,T
0,28.04.2020 23:30,8.0
1,28.04.2020 23:00,8.0
2,28.04.2020 22:30,8.0
3,28.04.2020 22:00,8.0
4,28.04.2020 21:30,8.0


Преобразовываем строковый параметр к виду mm-yyyy и группируем по месяцам. Получаем среднемесячную температуру.

In [6]:
weather_temp['local time Perm'] = weather_temp['local time Perm'].str.extract('(\d{2}.\d{2}.\d{4})')
weather_temp['local time Perm'] = weather_temp['local time Perm'].str.replace(r'^...','').str.replace('.','-')

In [7]:
weather = weather_temp.groupby('local time Perm').agg({'T': 'mean'}).reset_index()
weather.head()

,local time Perm,T
0,01-2018,-11.211210
1,01-2019,-11.163823
2,01-2020,-6.307012
3,02-2018,-12.592873
4,02-2019,-9.677711


Добавляем среднемесячную температуру в датасет supply

In [8]:
supply = supply_temp.merge(weather, how = 'left', left_on='Period', right_on='local time Perm')
supply.drop(['local time Perm'],axis=1, inplace=True)
supply.head()

,Method,Consumer,Building,Supply,Status,Period,T
0,ЛС-СДСД,66700000233,дача,0.0,неоспар.,01-2019,-11.163823
1,ЛС-СДСД,66700000410,дача,0.0,неоспар.,01-2019,-11.163823
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,неоспар.,01-2019,-11.163823
3,ЛС-СДСД,66700000557,дача,2.0,неоспар.,01-2019,-11.163823
4,ЛС-СДСД,66700000575,дача,21.0,неоспар.,01-2019,-11.163823


1.3. Загружаем данные по потребителям: количество комнат и количество проживающих

In [9]:
%%time
dirName = './Data/'
name = 'RoomsPeoples.xlsx'
fullname = os.path.join(dirName, name) # получаем полное имя
if os.path.isfile(fullname):
    RoomsPeoples = pd.read_excel(fullname)
    print ("Обработан файл:", fullname, " - загружено строк:", len(RoomsPeoples))
else: 
    print ("Ошибка загрузки:", fullname)

Обработан файл: ./Data/RoomsPeoples.xlsx  - загружено строк: 397671
Wall time: 23 s


In [10]:
RoomsPeoples.head()

,Consumer,Rooms,Peoples
0,62800121655,1.0,1.0
1,62840754350,1.0,1.0
2,00003,1.0,1.0
3,61007104593,1.0,1.0
4,61009048247,NaN,1.0


Добавляем информацию в датасет supply

In [11]:
supply = supply.merge(RoomsPeoples, how = 'left', left_on='Consumer', right_on='Consumer')
supply.head()

,Method,Consumer,Building,Supply,Status,Period,T,Rooms,Peoples
0,ЛС-СДСД,66700000233,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0
1,ЛС-СДСД,66700000410,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,неоспар.,01-2019,-11.163823,1.0,1.0
3,ЛС-СДСД,66700000557,дача,2.0,неоспар.,01-2019,-11.163823,1.0,1.0
4,ЛС-СДСД,66700000575,дача,21.0,неоспар.,01-2019,-11.163823,1.0,1.0


1.4. Загружаем данные по категориям потребителей из каталога /Data/Group/

In [12]:
%%time
dirName = './Data/Group/'
group_all = []
for name in os.listdir(dirName):
    fullname = os.path.join(dirName, name) # получаем полное имя
    if os.path.isfile(fullname):
        dfg = pd.read_excel(fullname)
        print ("Обработан файл:", fullname, " - загружено строк:", len(dfg))
        group_all.append(dfg)
group_temp = pd.concat(group_all,axis=0,ignore_index=True)
print("Всего строк загружено: ", len(group_temp))

Обработан файл: ./Data/Group/bes_filtered.xlsx  - загружено строк: 47614
Обработан файл: ./Data/Group/ces_filtered.xlsx  - загружено строк: 103516
Обработан файл: ./Data/Group/chaes_filtered.xlsx  - загружено строк: 83823
Обработан файл: ./Data/Group/chues_filtered.xlsx  - загружено строк: 28754
Обработан файл: ./Data/Group/kues_filtered.xlsx  - загружено строк: 80955
Обработан файл: ./Data/Group/oches_filtered.xlsx  - загружено строк: 63447
Обработан файл: ./Data/Group/pges_filtered.xlsx  - загружено строк: 101690
Обработан файл: ./Data/Group/ses_filtered.xlsx  - загружено строк: 42740
Всего строк загружено:  552539
Wall time: 21.2 s


In [23]:
group_temp.head()

,Consumer,Group
0,66700334153,Население сельское
1,66400019149,Население городское
2,66400042245,Население сельское
3,66400042147,Население сельское
4,66400042147,Население сельское


Удаляем дубликаты

In [25]:
group_temp.drop_duplicates(keep='first', inplace=True)
print(len(group_temp))

346300


Добавляем информацию в датасет supply

In [32]:
#supply.drop('Group', axis=1, inplace=True)
supply.head()

,Method,Consumer,Building,Supply,Status,Period,T,Rooms,Peoples
0,ЛС-СДСД,66700000233,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0
1,ЛС-СДСД,66700000410,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,неоспар.,01-2019,-11.163823,1.0,1.0
3,ЛС-СДСД,66700000557,дача,2.0,неоспар.,01-2019,-11.163823,1.0,1.0
4,ЛС-СДСД,66700000575,дача,21.0,неоспар.,01-2019,-11.163823,1.0,1.0


In [35]:
group_temp['Consumer'] = group_temp['Consumer'].astype(str)
supply = supply.merge(group_temp, how = 'left', left_on='Consumer', right_on='Consumer')
supply.head()

,Method,Consumer,Building,Supply,Status,Period,T,Rooms,Peoples,Group
0,ЛС-СДСД,66700000233,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0,Население сельское
1,ЛС-СДСД,66700000410,дача,0.0,неоспар.,01-2019,-11.163823,1.0,1.0,Население сельское
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,неоспар.,01-2019,-11.163823,1.0,1.0,Население сельское
3,ЛС-СДСД,66700000557,дача,2.0,неоспар.,01-2019,-11.163823,1.0,1.0,Население сельское
4,ЛС-СДСД,66700000575,дача,21.0,неоспар.,01-2019,-11.163823,1.0,1.0,Население сельское


### Этап 2. Обработка данных

2.1. Удаляем дубликаты

In [37]:
supply.drop_duplicates(keep='first', inplace=True)
print(len(supply))

5540202


2.2. Удаляем все строки со статусом Status = "оспар.", потом удаляем сам столбец за ненадобностью (он используется при расчетах полезного отпуска)

In [39]:
supply = supply.loc[supply['Status']!='оспар.']
print(len(supply))

5492567


In [40]:
supply.drop('Status', axis=1, inplace=True)
supply.head()

,Method,Consumer,Building,Supply,Period,T,Rooms,Peoples,Group
0,ЛС-СДСД,66700000233,дача,0.0,01-2019,-11.163823,1.0,1.0,Население сельское
1,ЛС-СДСД,66700000410,дача,0.0,01-2019,-11.163823,1.0,1.0,Население сельское
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,01-2019,-11.163823,1.0,1.0,Население сельское
3,ЛС-СДСД,66700000557,дача,2.0,01-2019,-11.163823,1.0,1.0,Население сельское
4,ЛС-СДСД,66700000575,дача,21.0,01-2019,-11.163823,1.0,1.0,Население сельское


2.3. Удаляем столбец "Period", т.к. он использовался только для добавления среднемесячной температуры

In [41]:
supply.drop('Period', axis=1, inplace=True)
supply.head()

,Method,Consumer,Building,Supply,T,Rooms,Peoples,Group
0,ЛС-СДСД,66700000233,дача,0.0,-11.163823,1.0,1.0,Население сельское
1,ЛС-СДСД,66700000410,дача,0.0,-11.163823,1.0,1.0,Население сельское
2,ЛС-СДСД,66700000431,част. ж/дом,2.0,-11.163823,1.0,1.0,Население сельское
3,ЛС-СДСД,66700000557,дача,2.0,-11.163823,1.0,1.0,Население сельское
4,ЛС-СДСД,66700000575,дача,21.0,-11.163823,1.0,1.0,Население сельское


2.4. Из столбца "Building" извлекаем данные о том, постоянно ли проживают по данному адресу (не дача) или нет. Добавляем столбец "Permanent_Residence", которое принимает значение 1 при постоянном проживании, и 0 в обратном случае.

In [42]:
supply['Building'].unique()

array(['дача', 'част. ж/дом', 'кварт.', 'баня', 'стр.площ.', 'гараж',
       'кв. малокв.', 'хоз.постр.', 'прочие хоз.', 'мнкв.дом', 'ТП',
       'Не опр.', 'прочие пром.', 'разрушенный', 'нежилой',
       'общест. объект', 'нежилое в мн.кв.', 'нежилая кварт.',
       'водокачка', 'религ. объект', 'общежитие', 'эл.отопл.', 'пилорама',
       'проч. объекты юр.лиц', 'мастерская', 'яма', 'сад',
       'мнкв.авар.дом', 'мнкв.ветх.дом'], dtype=object)

In [50]:
def Permanent_Residence(b):
    perm_list = ['кварт.','част. ж/дом','кв. малокв.','мнкв.дом','общежитие', 'эл.отопл.']
    if b in perm_list:
        return 1
    else:
        return 0

In [51]:
supply['Permanent_Residence'] = supply['Building'].apply(Permanent_Residence)

Удаляем столбец "Building"

In [54]:
supply.drop('Building', axis=1, inplace=True)
supply.head()

,Method,Consumer,Supply,T,Rooms,Peoples,Group,Permanent_Residence
0,ЛС-СДСД,66700000233,0.0,-11.163823,1.0,1.0,Население сельское,0
1,ЛС-СДСД,66700000410,0.0,-11.163823,1.0,1.0,Население сельское,0
2,ЛС-СДСД,66700000431,2.0,-11.163823,1.0,1.0,Население сельское,1
3,ЛС-СДСД,66700000557,2.0,-11.163823,1.0,1.0,Население сельское,0
4,ЛС-СДСД,66700000575,21.0,-11.163823,1.0,1.0,Население сельское,0


2.5. Из столбца "Group" извлекаем данные о том, потребитель сельский или городской. 
Добавляем столбец "City", которое принимает значение 1 при городском расположении, и 0 в обратном случае.

In [56]:
supply['Group'].unique()

array(['Население сельское', 'Население городское', nan,
       'Приравненные к населению (село)',
       'Приравненные к населению (город)', 'Не определено',
       'Непромышленные потребители', 'Промышленные потребители'],
      dtype=object)

In [57]:
def City_Residence(b):
    perm_list = ['Население городское','Приравненные к населению (город)','Непромышленные потребители', 'Промышленные потребители']
    if b in perm_list:
        return 1
    else:
        return 0

In [58]:
supply['City'] = supply['Group'].apply(City_Residence)

Удаляем столбец "Group"

In [59]:
supply.drop('Group', axis=1, inplace=True)
supply.head()

,Method,Consumer,Supply,T,Rooms,Peoples,Permanent_Residence,City
0,ЛС-СДСД,66700000233,0.0,-11.163823,1.0,1.0,0,0
1,ЛС-СДСД,66700000410,0.0,-11.163823,1.0,1.0,0,0
2,ЛС-СДСД,66700000431,2.0,-11.163823,1.0,1.0,1,0
3,ЛС-СДСД,66700000557,2.0,-11.163823,1.0,1.0,0,0
4,ЛС-СДСД,66700000575,21.0,-11.163823,1.0,1.0,0,0


2.6. Из столбца "Consumer" удаляем всех потребителей, у которых договор не состоит из одних цифр. 
Эти договоры не имеют отношение к населению.

In [61]:
supply[supply['Consumer'].str.contains('\D+',flags=re.IGNORECASE, regex=True)]

,Method,Consumer,Supply,T,Rooms,Peoples,Permanent_Residence,City
2265,ЛС-КПП,ОДУ У Беляева 26,0.0,-11.163823,NaN,NaN,1,0
9654,ЛС-КПБ,МКД ЮГ без УК,0.0,-11.163823,2.0,6.0,0,1
12637,ЛС-НОРМ,МКД СЕВЕР (не определено),0.0,-11.163823,8.0,64.0,0,1
14744,ЛС-НОРМ,МКД СЕВЕР (непосредственное),0.0,-11.163823,2.0,2.0,0,1
15424,ЛС-НОРМ,МКД ЮГ без УК,0.0,-11.163823,2.0,6.0,0,1
...,...,...,...,...,...,...,...,...
5545312,ЛС-ЭКП,65300004805,1250.0,0.552076,8.0,4.0,1,0
5560620,ЛС-СДСД,3+ОДПУ,1951.0,0.552076,16.0,16.0,1,1
5560692,ЛС-СДСД,3836 ОДПУ,293.0,0.552076,0.0,3.0,1,1
5594125,ЛС-НОРМ,для з/д,145.0,0.552076,12.0,17.0,1,1


In [63]:
supply = supply[~supply['Consumer'].str.contains('\D+',flags=re.IGNORECASE, regex=True)]

2.7. Проверяем на пустые значения и заполняем данными

In [65]:
supply.isnull().sum()

Method                      0
Consumer                    0
Supply                      0
T                           0
Rooms                  196913
Peoples                189959
Permanent_Residence         0
City                        0
dtype: int64

In [66]:
supply.describe()

,Supply,T,Rooms,Peoples,Permanent_Residence,City
count,5.488323e+06,5.488323e+06,5.291410e+06,5.298364e+06,5.488323e+06,5.488323e+06
mean,2.385289e+02,2.329030e+00,1.575323e+00,2.123765e+00,8.560313e-01,2.752804e-01
std,8.715553e+02,9.297464e+00,1.170233e+00,1.659900e+00,3.510581e-01,4.466555e-01
min,-4.572060e+05,-1.116382e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.300000e+01,-6.255830e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
50%,1.420000e+02,5.520762e-01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
75%,2.500000e+02,1.340122e+01,2.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00
max,3.280420e+05,1.713523e+01,5.100000e+01,7.700000e+01,1.000000e+00,1.000000e+00


Для поля "Rooms" ставим среднее округленное значение 2.0, для поля "Peoples" значение 2.0

In [70]:
supply.fillna(value={'Rooms': 2, 'Peoples': 2}, inplace=True)
supply.head()

,Method,Consumer,Supply,T,Rooms,Peoples,Permanent_Residence,City
0,ЛС-СДСД,66700000233,0.0,-11.163823,1.0,1.0,0,0
1,ЛС-СДСД,66700000410,0.0,-11.163823,1.0,1.0,0,0
2,ЛС-СДСД,66700000431,2.0,-11.163823,1.0,1.0,1,0
3,ЛС-СДСД,66700000557,2.0,-11.163823,1.0,1.0,0,0
4,ЛС-СДСД,66700000575,21.0,-11.163823,1.0,1.0,0,0


In [71]:
supply.isnull().sum()

Method                 0
Consumer               0
Supply                 0
T                      0
Rooms                  0
Peoples                0
Permanent_Residence    0
City                   0
dtype: int64

Теперь создаем копию датасета под именем ds_work, с ним работаем дальше. А в датасете supply остается полная копия данных, к которым можно вернуться, если надо будет изменять модель и добавлять новые признаки.

In [72]:
ds_work = supply.copy()
ds_work.head()

,Method,Consumer,Supply,T,Rooms,Peoples,Permanent_Residence,City
0,ЛС-СДСД,66700000233,0.0,-11.163823,1.0,1.0,0,0
1,ЛС-СДСД,66700000410,0.0,-11.163823,1.0,1.0,0,0
2,ЛС-СДСД,66700000431,2.0,-11.163823,1.0,1.0,1,0
3,ЛС-СДСД,66700000557,2.0,-11.163823,1.0,1.0,0,0
4,ЛС-СДСД,66700000575,21.0,-11.163823,1.0,1.0,0,0


Удаляем из датасета ds_work столбец Method, т.к. этот столбец будет нужен для более тонкой настройки модели. Если прямой подход "в лоб" не сработает.

In [73]:
ds_work.drop('Method', axis=1, inplace=True)
ds_work.head()

,Consumer,Supply,T,Rooms,Peoples,Permanent_Residence,City
0,66700000233,0.0,-11.163823,1.0,1.0,0,0
1,66700000410,0.0,-11.163823,1.0,1.0,0,0
2,66700000431,2.0,-11.163823,1.0,1.0,1,0
3,66700000557,2.0,-11.163823,1.0,1.0,0,0
4,66700000575,21.0,-11.163823,1.0,1.0,0,0


### Вопросы на подумать: 
#### 1. стоит ли оставлять в модели поле Consumer (преобразовав его в целый тип)?
#### 2. проверка на качество данных в признаках, стоит ли отсеять, например по 10% сверху и снизу? 

Достаточно ли для модели будет всего 5 признаков (T,Rooms,Peoples,Permanent_Residence,City), чтобы получить нормальный результат? 

In [74]:
ds_work.drop('Consumer', axis=1, inplace=True)
ds_work.head()

,Supply,T,Rooms,Peoples,Permanent_Residence,City
0,0.0,-11.163823,1.0,1.0,0,0
1,0.0,-11.163823,1.0,1.0,0,0
2,2.0,-11.163823,1.0,1.0,1,0
3,2.0,-11.163823,1.0,1.0,0,0
4,21.0,-11.163823,1.0,1.0,0,0


### Этап 3. Применение модели Линейной регрессии

In [76]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(ds_work[['T', 'Rooms', 'Peoples', 'Permanent_Residence','City']], ds_work['Supply'], test_size=0.3)

In [78]:
model = LinearRegression()

In [79]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Оценим качество модели

In [82]:
r_train = model.score(X_train, y_train)
r_test = model.score(X_test, y_test)
print('Качество модели на тестовых данных:', r_test, '\nКачество модели на тренировочных данных:', r_train)

Качество модели на тестовых данных: 0.003642565960207844 
Качество модели на тренировочных данных: 0.0038075644351687203


### Абсолютно неудовлетворительное качество модели

In [121]:
# df.to_excel('results.xlsx')